In [1]:
import numpy as np
import trimesh
import scipy

In [18]:
plane_mesh = trimesh.load_mesh("data/experiments/plane.obj")
cube_mesh = trimesh.load_mesh("data/experiments/cube.obj")
rectangle_mesh = trimesh.load_mesh("data/experiments/rectangle.obj")
human_mesh = trimesh.load_mesh("data/experiments/meshes_squat1_mesh_0010.obj")
try_mesh = trimesh.load_mesh("data/experiments/try.obj")

In [17]:

def filter_humphrey(mesh,alpha=0.1,beta=0.5,iterations=10,laplacian_operator=None):
    # get mesh vertices as vanilla numpy array
    vertices = mesh.vertices.copy().view(np.ndarray)
    # save original unmodified vertices
    original = vertices.copy()

    # run through iterations of filter
    for _index in range(iterations):
        vert_q = vertices.copy()
        vertices = laplacian_operator.dot(vertices)
        vert_b = vertices - (alpha * original + (1.0 - alpha) * vert_q)
        vertices -= (beta * vert_b + (1.0 - beta) *
                     laplacian_operator.dot(vert_b))

    # assign modified vertices back to mesh
    mesh.vertices = vertices
    return mesh

def filter_taubin(mesh,laplacian_operator):
 
    lamb = 0.5
    nu = 0.5
    iterations = 2

    # get mesh vertices as vanilla numpy array
    vertices = mesh.vertices.copy().view(np.ndarray)

    # run through multiple passes of the filter
    for index in range(iterations):
        # print(vertices)
        # do a sparse dot product on the vertices
        dot = laplacian_operator.dot(vertices) - vertices
        # alternate shrinkage and dilation
        if index % 2 == 0:
            vertices += lamb * dot
        else:
            vertices -= nu * dot

    # assign updated vertices back to mesh
    mesh.vertices = vertices
    return mesh

In [12]:
human_mesh.show()

In [13]:
import robust_laplacian
laplacian_operator = trimesh.smoothing.laplacian_calculation(human_mesh)
# laplacian_operator_robin = cotangent_weights(try_mesh.vertices, try_mesh.faces)
L, M = robust_laplacian.mesh_laplacian(np.array(human_mesh.vertices), np.array(human_mesh.faces), mollify_factor=0)
# print(L.toarray())

In [23]:
L, M = robust_laplacian.mesh_laplacian(np.array(human_mesh.vertices), np.array(human_mesh.faces), mollify_factor=0)
print(L)


  (0, 0)	4.553889802842072
  (2043, 0)	-0.13816021457969946
  (2667, 0)	-1.9683189627322983
  (5090, 0)	-0.08971639151995342
  (5536, 0)	-1.2194848568943144
  (9069, 0)	-1.138209377115806
  (1, 1)	4.100248577805049
  (2162, 1)	-0.7555814602147538
  (3190, 1)	-0.22667582096355465
  (3290, 1)	-1.0980282060738604
  (5563, 1)	-1.2238775589418394
  (6232, 1)	-0.564404969888201
  (7203, 1)	-0.23168056172283996
  (2, 2)	4.634021370376967
  (1904, 2)	-0.10130816862905039
  (5019, 2)	-0.8669737096138805
  (6417, 2)	-0.4553379144518034
  (7479, 2)	-0.7593680469678974
  (9129, 2)	-0.23867902969715799
  (9257, 2)	-2.2123545010171766
  (3, 3)	4.072889176962577
  (1133, 3)	-0.3989541163732053
  (4600, 3)	-0.5039805506461599
  (4868, 3)	-0.052553897198197475
  (5113, 3)	-0.5947732750698632
  :	:
  (5660, 9998)	-0.5158880913113877
  (8901, 9998)	-0.11160611822344725
  (9998, 9998)	4.731377746604702
  (99, 9999)	-0.21442448721376833
  (438, 9999)	-0.2945809872275375
  (864, 9999)	-0.3150821672044424
  

3.602879703925146e+16

In [231]:
  
import numpy as np
import scipy.sparse as sparse


def cotangent_weights(vertices, faces):
    """
    Compute the cotengenant weights matrix for mesh laplacian.
    Entry (i,i) is 1/6 of the sum of the area of surrounding triangles
    Entry (i,j) is 1/12 of the sum of the area of triangles using edge (i,j)
    Parameters
    -----------------------------
    vertices   : (n,3) array of vertices coordinates
    faces      : (m,3) array of vertex indices defining faces
    faces_area : (m,) - Optional, array of per-face area
    Output
    -----------------------------
    A : (n,n) sparse area matrix
    """
    N = vertices.shape[0]

    v1 = vertices[faces[:,0]]  # (m,3)
    v2 = vertices[faces[:,1]]  # (m,3)
    v3 = vertices[faces[:,2]]  # (m,3)

    # Edge lengths indexed by opposite vertex
    u1 = v3 - v2
    u2 = v1 - v3
    u3 = v2 - v1

    L1 = np.linalg.norm(u1,axis=1)  # (m,)
    L2 = np.linalg.norm(u2,axis=1)  # (m,)
    L3 = np.linalg.norm(u3,axis=1)  # (m,)

    # Compute cosine of angles
    A1 = np.einsum('ij,ij->i', -u2, u3) / (L2*L3)  # (m,)
    A2 = np.einsum('ij,ij->i', u1, -u3) / (L1*L3)  # (m,)
    A3 = np.einsum('ij,ij->i', -u1, u2) / (L1*L2)  # (m,)

    # Use cot(arccos(x)) = x/sqrt(1-x^2)
    I = np.concatenate([faces[:,0], faces[:,1], faces[:,2]])
    J = np.concatenate([faces[:,1], faces[:,2], faces[:,0]])
    S = np.concatenate([A3,A1,A2])
    S = 0.5 * S / np.sqrt(1-S**2)

    In = np.concatenate([I, J, I, J])
    Jn = np.concatenate([J, I, I, J])
    Sn = np.concatenate([-S, -S, S, S])
    W = sparse.coo_matrix((Sn, (In, Jn)), shape=(N, N)).tocsc()
    print("Sn: ",Sn)
    print("W: ", W)
    print("In: ", In)
    print("IJn: ", Jn)
    return W

def laplacian_calculation(mesh, equal_weight=True, pinned_vertices=[]):
    """
    Calculate a sparse matrix for laplacian operations.
    Parameters
    -------------
    mesh : trimesh.Trimesh
      Input geometry
    equal_weight : bool
      If True, all neighbors will be considered equally
      If False, all neighbors will be weighted by inverse distance
    Returns
    ----------
    laplacian : scipy.sparse.coo.coo_matrix
      Laplacian operator
    """
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors

    # if a node is pinned, it will average his coordinates by himself
    # in practice it will not move
    for i in pinned_vertices:
        neighbors[i] = [i]

    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    if equal_weight:
        # equal weights for each neighbor
        data = np.concatenate([[1.0 / len(n)] * len(n)
                               for n in neighbors])
    else:
        # umbrella weights, distance-weighted
        # use dot product of ones to replace array.sum(axis=1)
        ones = np.ones(3)
        # the distance from verticesex to neighbors
        norms = [
            1.0 / np.maximum(1e-6, np.sqrt(np.dot(
                (vertices[i] - vertices[n]) ** 2, ones)))
            for i, n in enumerate(neighbors)]
        # normalize group and stack into single array
        data = np.concatenate([i / i.sum() for i in norms])
        # data = np.concatenate(norms)

    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix


In [90]:
from sklearn.preprocessing import normalize
verts, faces = np.array(rectangle_mesh.vertices), np.array(rectangle_mesh.faces)
robin = cotangent_weights(verts, faces)
robin_copy = robin.toarray().copy()
# np.fill_diagonal(robin_copy, 0)
trimesh_laplace = laplacian_calculation(rectangle_mesh, equal_weight=False).toarray()
# norm2 = normalize(robin_copy[:,1], axis=0).ravel()
# data = np.array([abs(i) / np.linalg.norm(i) for i in robin_copy])
# data
# robin_copy

In [139]:
verts, faces = np.array(man_mesh.vertices), np.array(man_mesh.faces)
robin = cotangent_weights(verts, faces)
robin_copy = robin.toarray().copy()
robin_normalized = np.array([row/-row[i] for i, row in enumerate(robin_copy)])
np.fill_diagonal(robin_normalized, 0)
# robin_normalized = robin_normalized.astype(np.float32)
# robin_normalized
# for i, row in enumerate(robin_copy):
#     print(row/row[i] )
#     break
# a = np.array([ 6.33277862e+03, -1.57933510e-04, -3.16638923e+03,0.00000000e+00, -3.16638923e+03,  0.00000000e+00,0.00000000e+00,  0.00000000e+00])
# a/a[0]


In [127]:
a = robin_normalized[0].tolist()
a

[0.0,
 0.48418275921351767,
 0.25790862039324125,
 -0.0,
 0.25790862039324114,
 -0.0,
 -0.0,
 -0.0]

In [194]:
robin_laplace = cotangent_weights(verts, faces).toarray()
L, M = robust_laplacian.mesh_laplacian(verts, faces)
trimesh_laplace = trimesh.smoothing.laplacian_calculation(cube_mesh, equal_weight=False).toarray()
# print("robin laplace")
# print(robin_laplace.toarray())
print("robust laplace")
L = L.toarray()
print(L)
print("trimesh laplace")
print(trimesh_laplace)


robust laplace
[[ 3.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  -1.00000000e+00  2.22044605e-16  2.22044605e-16  0.00000000e+00]
 [-1.00000000e+00  3.00000000e+00  2.22044605e-16 -1.00000000e+00
   0.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  2.22044605e-16  3.00000000e+00 -1.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -1.00000000e+00  3.00000000e+00
   0.00000000e+00  2.22044605e-16  2.22044605e-16 -1.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.00000000e+00 -1.00000000e+00 -1.00000000e+00  2.22044605e-16]
 [ 2.22044605e-16 -1.00000000e+00  0.00000000e+00  2.22044605e-16
  -1.00000000e+00  3.00000000e+00  0.00000000e+00 -1.00000000e+00]
 [ 2.22044605e-16  0.00000000e+00 -1.00000000e+00  2.22044605e-16
  -1.00000000e+00  0.00000000e+00  3.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.0

In [43]:
from scipy.sparse import coo_matrix, eye
from scipy.spatial import distance

def get_cotangent_weight(i,neighbor,neighbors,norms, KNN):
    def get_k_h(i,j):
        list1, list2 = neighbors[i], neighbors[j]
        #if there is morethan 2 intersection choose the first 2 points
        intersection = list(set(list1).intersection(list2))
        try:
            k,h = intersection
        except:
            k,h = intersection[:2]
            # print(intersection)
            # print(i,j)
            # print('k,h', )

        return k,h
    def get_distance(i,j,k,h):
        j_idx = neighbors[i].index(j)
        lij = norms[i][j_idx]

        k_idx = neighbors[j].index(k)
        ljk = norms[j][k_idx]

        i_idx = neighbors[k].index(i)
        lki = norms[k][i_idx]

        h_idx = neighbors[j].index(h)
        ljh = norms[j][h_idx]

        i_idx = neighbors[h].index(i)
        lhi = norms[h][i_idx]

        return lij, ljk, lki, ljh, lhi 

    result = []
    for j in KNN:
        #if j in neighbor find Cotangent weight else assign 0
        wij = 0
        if j in neighbor:
            
            k,h  = get_k_h(i,j)
            lij, ljk, lki, ljh, lhi = get_distance(i,j,k,h)

            s_ijk = (lij + ljk + lki)/2
            A_ijk = 8 *  np.sqrt(s_ijk * ( s_ijk - lij) * ( s_ijk- ljk) * ( s_ijk - lki))

            s_ijh = (lij + ljh + lhi)/2
            A_ijh = 8 *  np.sqrt(s_ijh * ( s_ijh - lij) * ( s_ijh- ljh) * ( s_ijh - lhi))

            
            wij = ((-lij**2 + ljk ** 2 + lki**2)/  A_ijk)  + ((-lij**2 + ljh ** 2 + lhi**2)/  A_ijh)
        
       
        result.append(wij)
        
    return result

def knn_cotangent_laplacian(mesh, k):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]



    # norms = [i / i.sum() for i in norms]
    data = []

    for i, KNN in enumerate(closest):
        neighbor = neighbors[i]
        weight = get_cotangent_weight(i,neighbor, neighbors,norms, KNN)
        data.append(weight)
        # create the sparse matrix
    
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])
    
   
    
    data = np.concatenate([i / np.array(i).sum() if np.array(i).sum()>0 else i for i in data])
    
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix
knn_laplace = knn_cotangent_laplacian(plane_mesh, 2)

In [212]:
plane_mesh.vertex_neighbors

[[1, 2], [0, 2, 3], [0, 1, 3], [1, 2]]

In [34]:
smoothed_mesh = trimesh.smoothing.filter_humphrey(mesh,alpha=0.1,beta=0.5,iterations=10,laplacian_operator=cotan_matrix)

In [132]:
import robust_laplacian
from plyfile import PlyData
import numpy as np
import polyscope as ps
import scipy.sparse.linalg as sla
import trimesh

verts, faces = np.array(plane_mesh.vertices), np.array(plane_mesh.faces)
# L, M = robust_laplacian.mesh_laplacian(verts, faces)

# # Compute some eigenvectors
n_eig = 3
evals, evecs = sla.eigsh(trimesh_laplace, n_eig, sigma=1e-10)
# W, A = cotangent_weights(verts, faces), dia_area_mat(verts, faces, faces_areas=None)
# evals, evecs = laplacian_spectrum(W, A, spectrum_size=200)
# evals, evecs = sla.eigsh(cotan_matrix, n_eig)
# # Visualize
ps.init()
ps_cloud = ps.register_surface_mesh("my mesh", verts, faces)
# ps_cloud = ps.register_point_cloud("my cloud", points)
for i in range(n_eig):
    ps_cloud.add_scalar_quantity("eigenvector_"+str(i), evecs[:,i], enabled=True)
ps.show()

In [160]:
lion_mesh = trimesh.load_mesh("data/new_data/noisy_mesh/chinese-lion_n1.obj")
man_mesh = trimesh.load_mesh("data/new_data/noisy_mesh/meshes_bouncing_mesh_0000.obj")

In [36]:
robin_normalized.max() #0.9247

0.9995733594503334

In [163]:
verts, faces = np.array(man_mesh.vertices), np.array(man_mesh.faces)
# L, M = robust_laplacian.mesh_laplacian(verts, faces)
trimesh_laplace = trimesh.smoothing.laplacian_calculation(man_mesh, equal_weight=False)
trimesh_laplace =trimesh_laplace.toarray()
# robin_laplace = cotangent_weights(verts, faces)
# print(L.toarray())

# robin_copy = robin_laplace.toarray().copy()
np.fill_diagonal(trimesh_laplace, 1)
# robin_normalized = np.array([abs(i) / np.linalg.norm(i) for i in robin_copy])


In [164]:
trimesh_laplace

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [166]:
smoothed_mesh = filter_humphrey(man_mesh,iterations=5,laplacian_operator=trimesh_laplace)


In [167]:
smoothed_mesh.show()

In [100]:
man_mesh.show()